In [3]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import cv2
import numpy as np

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 12  # Your number of classes
cfg.MODEL.WEIGHTS = "model_final.pth"  # Path to your trained model weights
cfg.MODEL.DEVICE = "cpu"  # Use CPU for inference
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # Set a threshold for detection confidence
predictor = DefaultPredictor(cfg)


[03/04 08:48:57 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from model_final.pth ...


/home/arshlaan/virtual_python/chess/lib/python3.11/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_loca

In [ ]:
def detect_chess_pieces(image):
    outputs = predictor(image)
    instances = outputs["instances"].to("cpu")
    boxes = instances.pred_boxes.tensor.numpy()
    classes = instances.pred_classes.numpy()
    scores = instances.scores.numpy()
    return boxes, classes, scores

def map_to_chessboard(image, boxes, classes):
    board_state = [[""] * 8 for _ in range(8)]
    height, width = image.shape[:2]
    for box, cls in zip(boxes, classes):
        x, y = (box[0] + box[2]) / 2, (box[1] + box[3]) / 2
        row = int(8 * y / height)
        col = int(8 * x / width)
        if 0 <= row < 8 and 0 <= col < 8:
            board_state[row][col] = class_names[cls]
    return board_state

def generate_fen(board_state):
    fen = ""
    for row in board_state:
        empty = 0
        for piece in row:
            if piece == "":
                empty += 1
            else:
                if empty > 0:
                    fen += str(empty)
                    empty = 0
                fen += piece
        if empty > 0:
            fen += str(empty)
        fen += "/"
    fen = fen[:-1] + " w - - 0 1"  # Assuming it's white's turn to move
    return fen

# Define class names (adjust based on your training)
class_names = {
    0: "K", 1: "Q", 2: "R", 3: "B", 4: "N", 5: "P",
    6: "k", 7: "q", 8: "r", 9: "b", 10: "n", 11: "p"
}


In [ ]:
cap = cv2.VideoCapture(0)  # Use 0 for webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    boxes, classes, scores = detect_chess_pieces(frame)
    board_state = map_to_chessboard(frame, boxes, classes)
    fen = generate_fen(board_state)

    # Draw bounding boxes and FEN on the frame
    for box, cls, score in zip(boxes, classes, scores):
        x1, y1, x2, y2 = box.astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{class_names[cls]}: {score:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.putText(frame, f"FEN: {fen}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow("Chess Board Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


/home/arshlaan/virtual_python/chess/lib/python3.11/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


KeyboardInterrupt: 

: 